# Imports

In [1]:
import sys
import nltk
import numpy as np
import pandas as pd
import pickle
# from helpers import *
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn import metrics
import os
from matplotlib import pyplot as plt
sys.path.append(".")
sys.path.append("..")

# Set Params

In [2]:
# Supported datasets:
# impact
# urgency
column_to_predict = "urgency"
text_columns = "title"
# Supported algorithms
# "SVM"
# "NB"
# "KNN"
# "D3"
classifier = "SVM"

# Grid Search is used to find hyperparameters
# Searching for hyperparameters is time consuming
use_grid_search = True

# Removes stop words from processed text
remove_stop_words = True

# Used with 'remove_stop_words' and defines language of stop words collection
stop_words_lang = 'english'

# Word stemming using NLTK
use_stemming = False

# If use_stemming == True, fit_prior should be set to False
fit_prior = True

# min_data_per_class is used to determine number of samples required for each class.
# Classes with less than that will be excluded from the dataset
# Default value is 1
min_data_per_class = 1  

# Supported samplers
# "over"
# "smote"
sampler = "smote"

# KNN cannot be used with Grid Search
if classifier == "KNN":
    use_grid_search = False

# Load Dataset

In [3]:
# loading dataset from dprep in Workbench    
# dfTickets = package.run('AllTickets.dprep', dataflow_idx=0) 

# loading dataset from csv
dfTickets = pd.read_csv(
    './datasets/all_tickets_2.csv',
    engine='python',
    dtype=str
)  

# Remove Poorly Represented Classes

In [4]:
# Removing rows related to classes represented by low amount of data
print("Shape of dataset before removing classes with less then " + str(min_data_per_class) + " rows: "+str(dfTickets.shape))
print("Number of classes before removing classes with less then " + str(min_data_per_class) + " rows: "+str(len(np.unique(dfTickets[column_to_predict]))))
bytag = dfTickets.groupby(column_to_predict).aggregate(np.count_nonzero)
tags = bytag[bytag.body > min_data_per_class].index
dfTickets = dfTickets[dfTickets[column_to_predict].isin(tags)]
print( "Shape of dataset after removing classes with less then " + str(min_data_per_class) + " rows: " + str(dfTickets.shape))
print("Number of classes after removing classes with less then " + str(min_data_per_class) + " rows: " + str(len(np.unique(dfTickets[column_to_predict]))))

Shape of dataset before removing classes with less then 1 rows: (1576, 5)
Number of classes before removing classes with less then 1 rows: 3
Shape of dataset after removing classes with less then 1 rows: (1576, 5)
Number of classes after removing classes with less then 1 rows: 3


# Split Dataset into Training and Testing Data

In [5]:
labelData = dfTickets[column_to_predict]
data = dfTickets[text_columns]

# Split dataset into training and testing data with 80:20 ratio
train_data, test_data, train_labels, test_labels = train_test_split(data, labelData, test_size=0.2)

# Extract Features from the Text

In [6]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        stemmer = SnowballStemmer("english", ignore_stopwords=True)
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

## Count Vectorizer

In [7]:
if remove_stop_words:
    count_vect = CountVectorizer(stop_words=stop_words_lang)
elif use_stemming:
    count_vect = StemmedCountVectorizer(stop_words=stop_words_lang)
else:
    count_vect = CountVectorizer()

vectorized_data = count_vect.fit_transform(train_data)
vectorized_data.shape

(1260, 1066)

## Use TF-IDF

In [8]:
tfidf = TfidfTransformer()
features = tfidf.fit_transform(vectorized_data)
features.shape

(1260, 1066)

# Handle Imbalance Dataset

In [9]:
if sampler == "over":
    imbl_samp = RandomOverSampler()
else:
    imbl_samp = SMOTE(random_state=2)

# Use A Pipeline to Preprocess Data and Train Classifier

In [10]:
if classifier == "NB":
    print("Training NB classifier")
    text_clf = Pipeline([
        ('vect', count_vect),
        ('tfidf', TfidfTransformer()),
#         ('samp', imbl_samp),
        ('clf', MultinomialNB(fit_prior=fit_prior))
    ])
    text_clf = text_clf.fit(train_data, train_labels)
elif classifier == "D3":
    print("Training D3 classifier")
    text_clf = Pipeline([
        ('vect', count_vect),
        ('tfidf', TfidfTransformer()),
#         ('samp', imbl_samp),
        ('clf', DecisionTreeClassifier())
    ])
    text_clf = text_clf.fit(train_data, train_labels)
elif classifier == "SVM":
    print("Training SVM classifier")
    text_clf = Pipeline([
        ('vect', count_vect),
        ('tfidf', TfidfTransformer()),
#         ('samp', imbl_samp),
        ('clf', SVC(kernel='linear'))
    ])
    text_clf = text_clf.fit(train_data, train_labels)
elif classifier == "KNN":
    print("Training KNN classifier")
    text_clf = Pipeline([
        ('vect', count_vect),
        ('tfidf', TfidfTransformer()),
#         ('samp', imbl_samp),
        ('clf', KNeighborsClassifier(n_neighbors = 3))
    ])
    text_clf = text_clf.fit(train_data, train_labels)

Training SVM classifier


# Save Model to Pickle

In [11]:
pickle.dump(text_clf, open('./pickle/xtremax/text_'+classifier+'_'+text_columns+'_'+column_to_predict+'_model.pickle',"wb"))

# Use GridSearchCV to Find the Best Params Set

In [12]:
# List of parameters for performance tuning
if use_grid_search:
    parameters = {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'tfidf__use_idf': (True, False)
    }
    # Create GS instance by passing the classifier, parameters and n_jobs=-1 (use multiple cores from user machine)
    gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
    gs_clf = gs_clf.fit(train_data, train_labels)

    # To see the best mean score and the params
    gs_clf.best_score_
    gs_clf.best_params_

# Save GSCV Model to Pickle

In [13]:
if use_grid_search:
    pickle.dump(text_clf, open('./pickle/xtremax/gs_'+classifier+'_'+text_columns+'_'+column_to_predict+'_model.pickle',"wb"))

# Evaluate the Model

In [14]:
print("Evaluating model")
# Score and evaluate model on test data using model without hyperparameter tuning
predicted = text_clf.predict(test_data)
prediction_acc = np.mean(predicted == test_labels)
print("Confusion matrix without GridSearch:")
print(metrics.confusion_matrix(test_labels, predicted))
print("Mean without GridSearch: " + str(prediction_acc))


# Score and evaluate model on test data using model with hyperparameter tuning
if use_grid_search:
    predicted = gs_clf.predict(test_data)
    prediction_acc = np.mean(predicted == test_labels)
    print("Confusion matrix with GridSearch:")
    print(metrics.confusion_matrix(test_labels, predicted))
    print("Mean with GridSearch: " + str(prediction_acc))

Evaluating model
Confusion matrix without GridSearch:
[[  8   7  31]
 [  4  20  47]
 [  0  22 177]]
Mean without GridSearch: 0.6487341772151899
Confusion matrix with GridSearch:
[[  8   8  30]
 [  4  17  50]
 [  0  25 174]]
Mean with GridSearch: 0.629746835443038


# Test Model with Inputs

## Read Input

In [15]:
dfTestTickets = pd.read_csv(
    './test data/testing_tickets.csv',
    dtype=str
)
test_input_data = dfTestTickets[text_columns]

## Predict without GridSearchCV

In [16]:
text_clf_model = pickle.load(open('./pickle/xtremax/text_'+classifier+'_'+text_columns+'_'+column_to_predict+'_model.pickle',"rb"))
prediction_input = text_clf_model.predict(test_input_data)

i = 0
for result in prediction_input:
    print("No:",i+2)
    print(text_columns,",")
    print(test_input_data.iloc[i])
    print("-"*30)
    print(column_to_predict,":", result)
    print("="*30)
    i+=1

No: 2
title ,
service unavailable
------------------------------
urgency : 3
No: 3
title ,
for us not working
------------------------------
urgency : 3
No: 4
title ,
not responding
------------------------------
urgency : 3
No: 5
title ,
performance issues and
------------------------------
urgency : 3
No: 6
title ,
connection down th floor
------------------------------
urgency : 3
No: 7
title ,
inaccessible
------------------------------
urgency : 3
No: 8
title ,
urgent cannot open files on
------------------------------
urgency : 3
No: 9
title ,
wireless network down
------------------------------
urgency : 3
No: 10
title ,
not accessible
------------------------------
urgency : 3
No: 11
title ,
certificate expired
------------------------------
urgency : 3
No: 12
title ,
oracle errors when logging
------------------------------
urgency : 3
No: 13
title ,
ad account change
------------------------------
urgency : 3
No: 14
title ,
conference codes needed
----------------------------

## Predict with GridSearchCV

In [17]:
if use_grid_search:
    gs_clf_model = pickle.load(open('./pickle/xtremax/gs_'+classifier+'_'+text_columns+'_'+column_to_predict+'_model.pickle',"rb"))
    prediction_input = gs_clf_model.predict(test_input_data)

    i = 0
    for result in prediction_input:
        print("No:",i+2)
        print(text_columns,",")
        print(test_input_data.iloc[i])
        print("-"*30)
        print(column_to_predict,":", result)
        print("="*30)
        i+=1

No: 2
title ,
service unavailable
------------------------------
urgency : 3
No: 3
title ,
for us not working
------------------------------
urgency : 3
No: 4
title ,
not responding
------------------------------
urgency : 3
No: 5
title ,
performance issues and
------------------------------
urgency : 3
No: 6
title ,
connection down th floor
------------------------------
urgency : 3
No: 7
title ,
inaccessible
------------------------------
urgency : 3
No: 8
title ,
urgent cannot open files on
------------------------------
urgency : 3
No: 9
title ,
wireless network down
------------------------------
urgency : 3
No: 10
title ,
not accessible
------------------------------
urgency : 3
No: 11
title ,
certificate expired
------------------------------
urgency : 3
No: 12
title ,
oracle errors when logging
------------------------------
urgency : 3
No: 13
title ,
ad account change
------------------------------
urgency : 3
No: 14
title ,
conference codes needed
----------------------------

# Plot the Confusion Matrix

In [1]:
# Ploting confusion matrix with 'seaborn' module
# Use below line only with Jupyter Notebook
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
mat = confusion_matrix(test_labels, predicted)
plt.figure(figsize=(4, 4))
sns.set()
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(test_labels),
            yticklabels=np.unique(test_labels))
plt.xlabel('true label')
plt.ylabel('predicted label')
# Save confusion matrix to outputs in Workbench
# plt.savefig(os.path.join('.', 'outputs', 'confusion_matrix.png'))
plt.show()

ModuleNotFoundError: No module named 'seaborn'

# Printing the Classification report

In [19]:
%matplotlib inline
from sklearn.metrics import classification_report
print(classification_report(test_labels, predicted,
                            target_names=np.unique(test_labels)))

              precision    recall  f1-score   support

           1       0.67      0.17      0.28        46
           2       0.34      0.24      0.28        71
           3       0.69      0.87      0.77       199

    accuracy                           0.63       316
   macro avg       0.56      0.43      0.44       316
weighted avg       0.60      0.63      0.59       316

